In [17]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

print(tf.__version__)


1.12.0


In [32]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [33]:
print('Training: {} labels: {}'.format(len(train_images),len(train_labels)))

Training: 25000 labels: 25000


In [34]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [35]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

In [36]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [38]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [39]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Train on 15000 samples, validate on 10000 samples
Epoch 1/40
15000/15000 [==============================] - 2s 106us/step - loss: 0.6924 - acc: 0.5625 - val_loss: 0.6911 - val_acc: 0.6296
Epoch 2/40
15000/15000 [==============================] - 1s 44us/step - loss: 0.6884 - acc: 0.7152 - val_loss: 0.6855 - val_acc: 0.7228
Epoch 3/40
15000/15000 [==============================] - 1s 44us/step - loss: 0.6798 - acc: 0.7519 - val_loss: 0.6745 - val_acc: 0.7478
Epoch 4/40
15000/15000 [==============================] - 1s 48us/step - loss: 0.6640 - acc: 0.7509 - val_loss: 0.6564 - val_acc: 0.7598
Epoch 5/40
15000/15000 [==============================] - 1s 46us/step - loss: 0.6398 - acc: 0.7796 - val_loss: 0.6299 - val_acc: 0.7710
Epoch 6/40
15000/15000 [==============================] - 1s 45us/step - loss: 0.6078 - acc: 0.7954 - val_loss: 0.5977 - val_acc: 0.7838
Epoch 7/40
15000/15000 [==============================] - 1s 49us/step - loss: 0.5698 - acc: 0.8109 - val_loss: 0.5617 - val_ac

In [43]:
results = model.evaluate(test_data, test_labels)

print(results)

25000/25000 [==============================] - 1s 32us/step
[0.3146596819782257, 0.87408]
